In [ ]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import time

np.random.seed(30)

# Load Excel data
file_path = "distance_matrice.xlsx"
transmission_distances_df = pd.read_excel(file_path, sheet_name="Transmission_Costs", index_col=0)
travel_distances_df = pd.read_excel(file_path, sheet_name="Travel_Costs", index_col=0)

# Define per-mile costs
transmission_cost_per_mile = 0.04  # $/kWh per mile
travel_cost_per_mile = 0.70        # $/mile

# Define sets
I = list(travel_distances_df.index)   # Charging station locations
J = list(travel_distances_df.columns) # Demand locations
K = list(transmission_distances_df.index) # Power substations

# Convert distance matrices to cost matrices
T = transmission_distances_df.to_numpy() * transmission_cost_per_mile  # Transmission costs
C = travel_distances_df.to_numpy() * travel_cost_per_mile              # Travel costs

# Define parameters
F = 20000  # Fixed cost to open a charging station
D = np.random.randint(20, 29, size=len(J))  # Demand at locations (in kWh)
S = np.random.uniform(0.8, 1.0, size=(len(I), len(J)))  # Max fraction demand coverage
P = np.random.randint(300, 400, size=len(K))  # Power substation capacity (in kWh)
Q = np.random.randint(90, 100, size=len(I))  # Charging station capacity (in kWh) =100

# Create Gurobi model
model = gp.Model("Fixed_Charge_Location_Problem")

# Set optimality gap to 0.8%
model.setParam(GRB.Param.MIPGap, 0.04)

# Decision Variables
x = model.addVars(I, vtype=GRB.BINARY, name="x")  # Open charging station
y = model.addVars(K, I, lb=0, ub=1, vtype=GRB.CONTINUOUS, name="y")  # Power supply fraction
z = model.addVars(I, J, lb=0, ub=1, vtype=GRB.CONTINUOUS, name="z")  # Demand met fraction

# Objective Function: Minimize total cost
model.setObjective(
    gp.quicksum(F * x[i] for i in I) +
    gp.quicksum(C[i_idx, j_idx] * z[i, j] for i_idx, i in enumerate(I) for j_idx, j in enumerate(J)) +
    gp.quicksum(T[k_idx, i_idx] * y[k, i] for k_idx, k in enumerate(K) for i_idx, i in enumerate(I)),
    GRB.MINIMIZE
)

# Constraints

# 1. Demand constraint
for j in J:
    model.addConstr(gp.quicksum(z[i, j] for i in I) == 1, name=f"Demand_{j}")

# 2. Coverage constraint
for i_idx, i in enumerate(I):
    for j_idx, j in enumerate(J):
        model.addConstr(z[i, j] <= S[i_idx, j_idx] * x[i], name=f"Coverage_{i}_{j}")

# 3. Power substation capacity constraint
for k_idx, k in enumerate(K):
    model.addConstr(gp.quicksum(y[k, i] * Q[i_idx] for i_idx, i in enumerate(I)) <= P[k_idx], name=f"Power_Capacity_{k}")

# 4. Charging station capacity constraint
for i_idx, i in enumerate(I):
    model.addConstr(gp.quicksum(z[i, j] * D[j_idx] for j_idx, j in enumerate(J)) <= Q[i_idx] * x[i], name=f"Station_Capacity_{i}")

# 5. Transmission demand balancing constraint
for i in I:
    model.addConstr(gp.quicksum(y[k, i] for k in K) >= gp.quicksum(z[i, j] for j in J), name=f"Balance_{i}")

# Solve the model with runtime measurement
start_time = time.time()
model.optimize()
end_time = time.time()

runtime = end_time - start_time
print(f"\nModel solved in {runtime:.4f} seconds")

# Extract results
result_x = {i: x[i].X for i in I}
result_y = {(k, i): y[k, i].X for k in K for i in I}
result_z = {(i, j): z[i, j].X for i in I for j in J}

# Display results
opened_stations = [i for i in I if result_x[i] > 0.5]
num_opened_stations = len(opened_stations)

print(f"\nNumber of Charging Stations Opened: {num_opened_stations}")
print(f"Charging Stations Opened: {opened_stations}")

print("\nDemand Coverage by Charging Stations:")
for i in opened_stations:
    covered_demand = sum(result_z[(i, j)] * D[j_idx] for j_idx, j in enumerate(J))
    print(f"  - Charging Station {i} covers {covered_demand:.2f} units of demand")

print("\nPower Supply Distribution to Charging Stations:")
utilized_substations = set()
for i in opened_stations:
    supplied_power = 0
    for k in K:
        power_supply = result_y[(k, i)] * Q[I.index(i)]
        if power_supply > 0:
            utilized_substations.add(k)
            supplied_power += power_supply
    print(f"  - Charging Station {i} is supplied {supplied_power:.2f} units of power")

print("\nUtilized Power Substations:")
num_utilized_substations = len(utilized_substations)
print(f"Power Substations Used: {list(utilized_substations)}")
print(f"Number of Power Substations Utilized: {num_utilized_substations}")
